## Import Libraries and Load Data

In [58]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from gensim.models import Word2Vec 
import matplotlib.pyplot as plt
%matplotlib inline
from nltk import word_tokenize
import warnings;
warnings.filterwarnings('ignore')
from collections import Counter 

In [59]:
df_raw = pd.read_excel('tnved-CIS-02.xls')

In [60]:
df_raw.head()

,TNVED,Name,Unit,FULL_TEXT
0,0101,"Лошади, ослы, мулы и лошаки живые:",NaN,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ:"
1,0101210000,––чистопородные племенные животные,шт,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ЛОШАДИ: ЧИ..."
2,010129,––прочие:,NaN,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ЛОШАДИ: ПР..."
3,0101291000,–––убойные,шт,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ЛОШАДИ: ПР..."
4,0101299000,–––прочие,шт,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ЛОШАДИ: ПР..."


In [61]:
df_raw.shape

(15927, 4)

The dataset contains 541,909 transactions. That is a pretty good number for us.

## Treat Missing Data

In [62]:
# check for missing values
df_raw.isnull().sum()

TNVED           0
Name           31
Unit         4375
FULL_TEXT      31
dtype: int64

<br>
Since we have sufficient data, we will drop all the rows with missing values.

In [63]:
# remove missing values
df_raw.dropna(inplace=True)

# again check missing values
df_raw.isnull().sum()

TNVED        0
Name         0
Unit         0
FULL_TEXT    0
dtype: int64

## Data Preparation

Let's convert the StockCode to string datatype.

In [64]:
df_raw['Cust_ID']= df_raw['TNVED'].astype(str)

Let's check out the number of unique customers in our dataset.

In [65]:
customers = df_raw["TNVED"].unique().tolist()
len(customers)

11552

In [66]:
import string
import nltk
import nltk
nltk.download('stopwords')
lst_stopwords = nltk.corpus.stopwords.words('russian')
lst_stopwords.append('прочие')
lst_stopwords.append('включая')
import re
lst_stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zhiti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [67]:
def clean_text(text, tokenizer, stopwords):
    """Pre-process text and generate tokens

    Args:
        text: Text to tokenize.

    Returns:
        Tokenized text.
    """
    text = str(text).lower()  # Lowercase words
    text = re.sub(r"\[(.*?)\]", "", text)  # Remove [+XYZ chars] in content
    text = re.sub(r"\s+", " ", text)  # Remove multiple spaces in content
    text = re.sub(r"\w+…|…", "", text)  # Remove ellipsis (and last word)
    text = re.sub(r"(?<=\w)-(?=\w)", " ", text)  # Replace dash between words
    text = re.sub(
        f"[{re.escape(string.punctuation)}]", "", text
    )  # Remove punctuation

    tokens = tokenizer(text)  # Get tokens from text
    tokens = [t for t in tokens if not t in lst_stopwords]  # Remove stopwords
    tokens = ["" if t.isdigit() else t for t in tokens]  # Remove digits
    tokens = [t for t in tokens if len(t) > 1]  # Remove short tokens
    return tokens

In [68]:
nltk.download('punkt')
text_columns = ['FULL_TEXT']

df = df_raw
df[ "FULL_TEXT"] = df[ "FULL_TEXT"].fillna(" ")

for col in text_columns:
    df[col] = df[col].astype(str)

# Create text column based on title, description, and content
df["text"] = df[text_columns].apply(lambda x: " | ".join(x), axis=1)
df["tokens"] = df["text"].map(lambda x: clean_text(x, word_tokenize, lst_stopwords))

#Remove duplicated after preprocessing
_, idx = np.unique(df["tokens"], return_index=True)
df = df.iloc[idx, :]

# Remove empty values
df = df.loc[df.tokens.map(lambda x: len(x) > 0), ["text", "tokens"]]

print(f"Original dataframe: {df_raw.shape}")
print(f"Pre-processed dataframe: {df.shape}")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zhiti\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Original dataframe: (11552, 7)
Pre-processed dataframe: (10783, 2)


In [69]:
df_raw

,TNVED,Name,Unit,FULL_TEXT,Cust_ID,text,tokens
1,0101210000,––чистопородные племенные животные,шт,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ЛОШАДИ: ЧИ...",0101210000,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ЛОШАДИ: ЧИ...","[лошади, ослы, мулы, лошаки, живые, лошади, чи..."
3,0101291000,–––убойные,шт,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ЛОШАДИ: ПР...",0101291000,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ЛОШАДИ: ПР...","[лошади, ослы, мулы, лошаки, живые, лошади, уб..."
4,0101299000,–––прочие,шт,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ЛОШАДИ: ПР...",0101299000,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ЛОШАДИ: ПР...","[лошади, ослы, мулы, лошаки, живые, лошади]"
5,0101300000,–ослы,шт,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ОСЛЫ",0101300000,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ОСЛЫ","[лошади, ослы, мулы, лошаки, живые, ослы]"
6,0101900000,–прочие,шт,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ПРОЧИЕ",0101900000,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ПРОЧИЕ","[лошади, ослы, мулы, лошаки, живые]"
...,...,...,...,...,...,...,...
15922,9702000000,"Подлинники гравюр, эстампов и литографий",шт,"ПОДЛИННИКИ ГРАВЮР, ЭСТАМПОВ И ЛИТОГРАФИЙ",9702000000,"ПОДЛИННИКИ ГРАВЮР, ЭСТАМПОВ И ЛИТОГРАФИЙ","[подлинники, гравюр, эстампов, литографий]"
15923,9703000000,Подлинники скульптур и статуэток из любых мате...,шт,ПОДЛИННИКИ СКУЛЬПТУР И СТАТУЭТОК ИЗ ЛЮБЫХ МАТЕ...,9703000000,ПОДЛИННИКИ СКУЛЬПТУР И СТАТУЭТОК ИЗ ЛЮБЫХ МАТЕ...,"[подлинники, скульптур, статуэток, любых, мате..."
15924,9704000000,"Марки почтовые или марки госпошлин, знаки почт...",–,"МАРКИ ПОЧТОВЫЕ ИЛИ МАРКИ ГОСПОШЛИН, ЗНАКИ ПОЧТ...",9704000000,"МАРКИ ПОЧТОВЫЕ ИЛИ МАРКИ ГОСПОШЛИН, ЗНАКИ ПОЧТ...","[марки, почтовые, марки, госпошлин, знаки, поч..."
15925,9705000000,Коллекции и предметы коллекционирования по зоо...,–,КОЛЛЕКЦИИ И ПРЕДМЕТЫ КОЛЛЕКЦИОНИРОВАНИЯ ПО ЗОО...,9705000000,КОЛЛЕКЦИИ И ПРЕДМЕТЫ КОЛЛЕКЦИОНИРОВАНИЯ ПО ЗОО...,"[коллекции, предметы, коллекционирования, зоол..."


In [70]:
# shuffle customer ID's
random.shuffle(customers)

# extract 90% of customer ID's
customers_train = [customers[i] for i in range(round(0.9*len(customers)))]

# split data into train and validation set
train_df = df_raw[df_raw['Cust_ID'].isin(customers_train)]
validation_df = df_raw[~df_raw['Cust_ID'].isin(customers_train)]

Let's create sequences of purchases made by the customers in the dataset for both the train and validation set.

In [71]:
# list to capture purchase history of the customers
purchases_train = []

# populate the list with the product codes
for i in tqdm(customers_train):
    temp = train_df[train_df["Cust_ID"] == i]["TNVED"].tolist()
    purchases_train.append(temp)

100%|███████████████████████████████████████████████████████████████████████████| 10397/10397 [00:17<00:00, 587.75it/s]


In [72]:
# list to capture purchase history of the customers
purchases_val = []

# populate the list with the product codes
for i in tqdm(validation_df['Cust_ID'].unique()):
    temp = validation_df[validation_df["Cust_ID"] == i]["TNVED"].tolist()
    purchases_val.append(temp)

100%|█████████████████████████████████████████████████████████████████████████████| 1155/1155 [00:01<00:00, 858.01it/s]


In [90]:
purchases_val[0]

['0101291000']

In [73]:
docs = df_raw["tokens"].values
tokenized_docs = df_raw["tokens"].values
vocab = Counter()
for token in tokenized_docs:
    vocab.update(token)

In [76]:
tokenized_docs[0]

['лошади',
 'ослы',
 'мулы',
 'лошаки',
 'живые',
 'лошади',
 'чистопородные',
 'племенные',
 'животные']

## Build word2vec Embeddings for Products

In [77]:
# train word2vec model
model = Word2Vec(window = 10, sg = 1, hs = 0,
                 negative = 10, # for negative sampling
                 alpha=0.03, min_alpha=0.0007,
                 seed = 14)

In [78]:
model.build_vocab(tokenized_docs)

In [79]:
model.train(tokenized_docs, total_examples=model.corpus_count, epochs=30, report_delay=1)

(6942146, 7586730)

In [20]:
# save word2vec model
model.save("word2vec_2.model")

As we do not plan to train the model any further, we are calling init_sims(), which will make the model much more memory-efficient.

In [80]:
model.init_sims(replace=True)

In [81]:
print(model)

Word2Vec(vocab=5710, vector_size=100, alpha=0.03)


Now we will extract the vectors of all the words in our vocabulary and store it in one place for easy access.

In [85]:
model.wv.key_to_index

{'кроме': 0,
 'содержащие': 1,
 'товарной': 2,
 'позиции': 3,
 'части': 4,
 'изделия': 5,
 'других': 6,
 'мас': 7,
 'аналогичные': 8,
 'менее': 9,
 'прочая': 10,
 'машины': 11,
 'мм': 12,
 'сахара': 13,
 'материалов': 14,
 'металлов': 15,
 'свежие': 16,
 'например': 17,
 'прочих': 18,
 'веществ': 19,
 'продукты': 20,
 'оборудование': 21,
 'прочий': 22,
 'другом': 23,
 'волокон': 24,
 'добавления': 25,
 'месте': 26,
 'средства': 27,
 'обработки': 28,
 'охлажденные': 29,
 'спирта': 30,
 'поименованные': 31,
 'средств': 32,
 'транспортных': 33,
 'аппаратура': 34,
 'подслащивающих': 35,
 'прочее': 36,
 'включенные': 37,
 'устройства': 38,
 'ткани': 39,
 'используемые': 40,
 'производные': 41,
 'добавок': 42,
 'соединения': 43,
 'кг': 44,
 'рыба': 45,
 'виде': 46,
 'стали': 47,
 'принадлежности': 48,
 'рыбы': 49,
 'мясо': 50,
 'товарных': 51,
 'содержанием': 52,
 'позиций': 53,
 'моторных': 54,
 'черных': 55,
 'способом': 56,
 'замороженные': 57,
 'смеси': 58,
 'виноградное': 59,
 'сусло': 

In [87]:
model.wv.index_to_key

['кроме',
 'содержащие',
 'товарной',
 'позиции',
 'части',
 'изделия',
 'других',
 'мас',
 'аналогичные',
 'менее',
 'прочая',
 'машины',
 'мм',
 'сахара',
 'материалов',
 'металлов',
 'свежие',
 'например',
 'прочих',
 'веществ',
 'продукты',
 'оборудование',
 'прочий',
 'другом',
 'волокон',
 'добавления',
 'месте',
 'средства',
 'обработки',
 'охлажденные',
 'спирта',
 'поименованные',
 'средств',
 'транспортных',
 'аппаратура',
 'подслащивающих',
 'прочее',
 'включенные',
 'устройства',
 'ткани',
 'используемые',
 'производные',
 'добавок',
 'соединения',
 'кг',
 'рыба',
 'виде',
 'стали',
 'принадлежности',
 'рыбы',
 'мясо',
 'товарных',
 'содержанием',
 'позиций',
 'моторных',
 'черных',
 'способом',
 'замороженные',
 'смеси',
 'виноградное',
 'сусло',
 'пород',
 'пряжи',
 'полученные',
 'готовые',
 'добавлением',
 'данной',
 'субпродукты',
 'имеющие',
 'spp',
 'примечании',
 'предназначенные',
 'производства',
 'подвергнутые',
 'исключением',
 'массой',
 'первичных',
 'соки',
 

In [95]:
model.wv.similar_by_vector("зажигалки")

[('сигаретные', 0.983447790145874),
 ('кремней', 0.974555492401123),
 ('фитилей', 0.9714259505271912),
 ('корневища', 0.7092963457107544),
 ('карманные', 0.6307641267776489),
 ('скреперы', 0.6220042705535889),
 ('локомотивы', 0.6173349618911743),
 ('газовые', 0.6128707528114319),
 ('аккумулирующие', 0.5921788811683655),
 ('батареи', 0.5863626003265381)]

In [103]:
test_vector1 = model.wv['зажигалки']
test_vector2 = model.wv['газовые']

sentence = [test_vector1, test_vector2]

TypeError: only integer scalar arrays can be converted to a scalar index

In [107]:
result = np.mean(sentence, axis=0)

In [116]:
recomended = model.wv.similar_by_vector(result)[:3]

labels = []
for label, chance in recomended:
    print(label)
    labels.append(label)

зажигалки
газовые
сигаретные


1        [лошади, ослы, мулы, лошаки, живые, лошади, чи...
3        [лошади, ослы, мулы, лошаки, живые, лошади, уб...
4              [лошади, ослы, мулы, лошаки, живые, лошади]
5                [лошади, ослы, мулы, лошаки, живые, ослы]
6                      [лошади, ослы, мулы, лошаки, живые]
                               ...                        
15922           [подлинники, гравюр, эстампов, литографий]
15923    [подлинники, скульптур, статуэток, любых, мате...
15924    [марки, почтовые, марки, госпошлин, знаки, поч...
15925    [коллекции, предметы, коллекционирования, зоол...
15926                        [антиквариат, возрастом, лет]
Name: tokens, Length: 11552, dtype: object